In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [1]:
import copy
import torch
import os
import time
import glob
import gc
from torchvision import transforms
from dataset import UDSegDataset
from torch.utils.data import DataLoader
from models import LinkNet34
from loss import LossMulti, MyLossMulti
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## training loop
w = 800
h = 544
batch_sz = 8

files = { x: [ft for ft in zip(
        glob.glob('data/'+x+'/CameraRGB/*.png'), 
        glob.glob('data/'+x+'/CameraSeg/*.png')
    )] for x in {'train', 'val'}}
data_transforms = {
    'train': transforms.Compose([
        lambda x: x[:544],
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        lambda x: x[:544],
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataloader = {x: 
              DataLoader(
                  UDSegDataset(files[x], data_transforms[x]), 
                  batch_size=8, 
                  shuffle=True, 
              ) 
              for x in {'train','val'}}

In [2]:
## training loop
w = 800
h = 544
batch_sz = 8

files = [ft for ft in zip(
        glob.glob('data/all/CameraRGB/*.png'), 
        glob.glob('data/all/CameraSeg/*.png')
    )] 
data_transforms = {
    'train': transforms.Compose([
        lambda x: x[:544],
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        lambda x: x[:544],
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataloader = DataLoader(
                  UDSegDataset(files, data_transforms['train']), 
                  batch_size=8, 
                  shuffle=True, 
              )

In [3]:
model = LinkNet34(num_classes=3, pretrained=True).cuda()
#model.load_state_dict(torch.load('./best.pt'))

In [4]:
#criterion = LossMulti(jaccard_weight=1, num_classes=3, class_weights=torch.tensor([.1, .65, .25], dtype=torch.float).cuda())
criterion = LossMulti(jaccard_weight=1, num_classes=3)
optimizer = optim.Adam(model.parameters())
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30], gamma=0.1)
num_epochs = 30
road_beta, car_beta = 0.5, 2

In [5]:
# train_model(model, criterion, optimizer, exp_lr_scheduler)
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)

#     scheduler.step()
    model.train()


    running_loss = 0.0
    running_corrects = 0.0
    eps = 1e-15
    road_prec, road_recall, car_prec, car_recall = 0,0,0,0
    for bi, (inputs, labels) in enumerate(dataloader):

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()


        running_loss += loss.item() * inputs.size(0)

        road_prec += (torch.sum((preds == 1) * (labels.data == 1)).item() / 
                          (torch.sum(preds == 1).item() + eps))
        road_recall += (torch.sum((preds == 1) * (labels.data == 1)).item() / 
                          (torch.sum(labels.data == 1).item() + eps))

        car_prec += (torch.sum((preds == 2) * (labels.data == 2)).item() / 
                          (torch.sum(preds == 2).item() + eps))
        car_recall += (torch.sum((preds == 2) * (labels.data == 2)).item() / 
                          (torch.sum(labels.data == 2).item() + eps))

#             print(('{} Loss: {:.4f}, car_re: {:.3f}, car_pre: {:.3f}, road_re: {:.3f}, road_pre: {:.4f}')
#                   .format(
#                       bi+1, running_loss/(bi+1), car_recall/(bi+1), car_prec/(bi+1), 
#                       road_recall/(bi+1), road_prec/(bi+1)))

    car_prec /= len(dataloader)
    car_recall /= len(dataloader)
    road_prec /= len(dataloader)
    road_recall /= len(dataloader)

    road_f1 = (1+road_beta**2) * ((road_prec*road_recall)/
                                      (road_beta**2*road_prec+road_recall+eps))
    car_f1 = (1+car_beta**2) * ((car_prec*car_recall)/
                                  (car_beta**2*car_prec+car_recall+eps))
    epoch_acc = (road_f1+car_f1)/2


    epoch_loss = running_loss / len(dataloader)

    print(('Loss: {:.4f}, car_re: {:.3f}, car_pre: {:.3f}, road_re: {:.3f}, ' + 
          'road_pre: {:.4f}, total_score: {:.4f}').format(
            epoch_loss, car_recall, car_prec, road_recall, road_prec, epoch_acc))

    best_acc = epoch_acc
    best_model_wts = copy.deepcopy(model.state_dict())
    torch.save(model.state_dict(), './model/epoch{}.pt'.format(epoch))
            

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
model.load_state_dict(best_model_wts)

Epoch 0/29
----------
Loss: 3.6046, car_re: 0.000, car_pre: 0.006, road_re: 0.598, road_pre: 0.5698, total_score: 0.2877

Epoch 1/29
----------
Loss: 1.8124, car_re: 0.181, car_pre: 0.207, road_re: 0.951, road_pre: 0.9461, total_score: 0.5663

Epoch 2/29
----------
Loss: 1.3479, car_re: 0.584, car_pre: 0.690, road_re: 0.959, road_pre: 0.9533, total_score: 0.7783

Epoch 3/29
----------
Loss: 1.1770, car_re: 0.619, car_pre: 0.734, road_re: 0.967, road_pre: 0.9615, total_score: 0.8009

Epoch 4/29
----------
Loss: 1.0936, car_re: 0.645, car_pre: 0.756, road_re: 0.968, road_pre: 0.9649, total_score: 0.8149

Epoch 5/29
----------
Loss: 1.1084, car_re: 0.620, car_pre: 0.759, road_re: 0.969, road_pre: 0.9637, total_score: 0.8040

Epoch 6/29
----------
Loss: 1.0420, car_re: 0.645, car_pre: 0.774, road_re: 0.972, road_pre: 0.9678, total_score: 0.8178

Epoch 7/29
----------
Loss: 1.0015, car_re: 0.646, car_pre: 0.788, road_re: 0.975, road_pre: 0.9707, total_score: 0.8207

Epoch 8/29
----------
Lo

In [ ]:
# train_model(model, criterion, optimizer, exp_lr_scheduler)
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)

    for phase in ['train', 'val']:
#     for phase in ['val']:
        if phase == 'train':
            scheduler.step()
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0.0
        eps = 1e-15
        road_prec, road_recall, car_prec, car_recall = 0,0,0,0
        for bi, (inputs, labels) in enumerate(dataloader[phase]):

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, dim=1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            
            running_loss += loss.item() * inputs.size(0)
            
            road_prec += (torch.sum((preds == 1) * (labels.data == 1)).item() / 
                              (torch.sum(preds == 1).item() + eps))
            road_recall += (torch.sum((preds == 1) * (labels.data == 1)).item() / 
                              (torch.sum(labels.data == 1).item() + eps))
            
            car_prec += (torch.sum((preds == 2) * (labels.data == 2)).item() / 
                              (torch.sum(preds == 2).item() + eps))
            car_recall += (torch.sum((preds == 2) * (labels.data == 2)).item() / 
                              (torch.sum(labels.data == 2).item() + eps))
            
#             print(('{} Loss: {:.4f}, car_re: {:.3f}, car_pre: {:.3f}, road_re: {:.3f}, road_pre: {:.4f}')
#                   .format(
#                       bi+1, running_loss/(bi+1), car_recall/(bi+1), car_prec/(bi+1), 
#                       road_recall/(bi+1), road_prec/(bi+1)))
        
        car_prec /= len(dataloader[phase])
        car_recall /= len(dataloader[phase])
        road_prec /= len(dataloader[phase])
        road_recall /= len(dataloader[phase])
        
        road_f1 = (1+road_beta**2) * ((road_prec*road_recall)/
                                          (road_beta**2*road_prec+road_recall+eps))
        car_f1 = (1+car_beta**2) * ((car_prec*car_recall)/
                                      (car_beta**2*car_prec+car_recall+eps))
        epoch_acc = (road_f1+car_f1)/2
        
        
        epoch_loss = running_loss / len(dataloader[phase])

        print(('{} Loss: {:.4f}, car_re: {:.3f}, car_pre: {:.3f}, road_re: {:.3f}, ' + 
              'road_pre: {:.4f}, total_score: {:.4f}').format(
                phase, epoch_loss, car_recall, car_prec, road_recall, road_prec, epoch_acc))
        
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            #torch.save(best_model_wts, './best.pt')
            

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
model.load_state_dict(best_model_wts)

In [ ]:
torch.save(best_model_wts, './new_best.pt')

In [ ]:
tdl = dataloader['train']

In [ ]:
imgs, masks = None, None
for i, m in tdl:
    imgs, masks = i, m
    break

In [ ]:
outputs = model(imgs)

In [ ]:
_, preds = torch.max(outputs, dim=1)

In [ ]:
plt.imshow(preds[0].cpu())

In [ ]:
plt.imshow(masks[0].cpu())

In [ ]:
import cv2

In [ ]:
img = imgs[0].cpu().numpy().transpose(1, 2, 0)
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img = std * img + mean
img = np.clip(img, 0, 1)
mask = masks[0].cpu().numpy()
my_mask = outputs[0].cpu().detach().numpy()

In [ ]:
my_mask = np.argmax(my_mask, axis=0).astype(np.float64)

In [ ]:
masked_img = np.copy(img)

In [ ]:
masked_img[(my_mask==1).nonzero()] = (0, 0, 1)
masked_img[(my_mask==2).nonzero()] = (0, 1, 0)

In [ ]:
plt.imshow(masked_img)

In [ ]:
weighted_img = cv2.addWeighted(img, .7, masked_img, .3, 0)

In [ ]:
plt.imshow(weighted_img)

In [ ]:
plt.imshow(np.pad(weighted_img, ((0, 56), (0, 0), (0,0)), 'constant', constant_values=(0)))